In [ ]:
import numpy as np
import itertools
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sys.path.append('/home/mi/thempel/code/information/')
import informant as inf

## Directed Information & Ising models
1. goal: Identify coupling qualitatively
2. goal: Set-up a driven Ising system and identify the "causal" spin

### Simple Ising model

In [ ]:
class OneDeeIsing(object):
    def __init__(self, nspins = 4, coupling = 1.0, invtemp = 1.):
    #initiate random state
        self._state = np.ones(nspins)
        for i,_s in enumerate(self._state):
            if np.random.rand()>0.5:
                self._state[i] = -1*self._state[i]

        self._nspins = nspins
        self._coupling = coupling
        self._beta = invtemp

        self._stayed = 0
  
    def tot_energy(self):
        return -self._coupling*(np.sum(self._state*np.roll(self._state, -1)) + np.sum(self._state*np.roll(self._state, 1)))

    def _energy_pos(self, i):
        return -self._coupling*(self._state[i-1]*self._state[i] + self._state[i]*self._state[(i+1)%self._nspins])

    def energy_pos(self, i):
        return self._energy_pos(i-1) +  self._energy_pos(i) + self._energy_pos((i+1)%self._nspins)

    def move(self, pos):
        prev_energy = self.tot_energy() #self.energy_pos(pos)
        __flip = np.random.choice([-1., 1.])
        self._state[pos] = __flip*self._state[pos]
        new_energy = self.tot_energy() #self.energy_pos(pos)
        if new_energy < prev_energy:
            pass
        else:
            dE = new_energy - prev_energy
            if np.random.rand()>np.exp(-dE*self._beta):
                self._state[pos] = __flip*self._state[pos]

    def sweep(self):
        _ris = np.random.randint(0, self._state.shape[0], self._state.shape[0])
        for i in _ris :
            self.move(i)

    def driven_sweep(self, index=0, staytime=10):
        self._state[index] = - self._state[index] if np.random.rand() > .92 else self._state[index]
        
        _ris = np.random.choice(list(range(0, index)) + list(range(index+1, self._state.shape[0])), self._state.shape[0])
        for i in _ris :
            self.move(i)

#### Ising standard variant, symmetric coupling

In [ ]:
Ising = OneDeeIsing(nspins = 13, coupling = .5, invtemp = 1.0)
states = Ising._state.reshape(-1,1)
energies = [Ising.tot_energy()]

for i in range(10000):
    Ising.sweep()
    states = np.hstack((states, Ising._state.reshape(-1,1)))
    energies.append(Ising.tot_energy())


fig, ax = plt.subplots(3,1, figsize=(10, 5))

ax[0].plot(energies)
ax[0].set_xlabel('Sweep count')
ax[0].set_ylabel('Total system energy')

ax[1].plot(states.mean(axis=0) )#32, histtype = 'step', log=True)
ax[1].set_ylabel('Effective magnetisation')
ax[1].set_xlabel('Sweep count')

_c = ax[2].imshow(states[:, :100])
ax[2].set_xlabel('Sweep count')
ax[2].set_ylabel('state of spins')
fig.colorbar(_c)

In [ ]:
plt.plot(states[0][:100])
plt.plot(states[1][:100])

In [ ]:
A, B = (states[1]==1).astype(int), (states[0]==1).astype(int)

In [ ]:
msmlags = range(1, 30, 2)

r = np.zeros((len(msmlags), 3))

for n, lag in enumerate(msmlags):
    est = inf.TransferEntropy(inf.MSMProbabilities(msmlag=lag, reversible=False, tmat_ck_estimate=False))
    est.symmetrized_estimate(A, B)
    r[n, :] = [est.d, est.r, inf.MutualInfoStationaryDistribution(est.p_estimator).estimate(A, B).m]

In [ ]:
fig, ax = plt.subplots()
for label, line in zip(['d', 'r', 'm'], r.T):
    if label == 'm':
        tax = ax.twinx()
        tax.plot(msmlags, line, '.-', label=label)
    else:
        ax.plot(msmlags, line, '.-', label=label, )

tax.set_ylim(.4, .5)
ax.set_xlabel('msm lagtime')
ax.legend()

In [ ]:
mmat = np.zeros((states.shape[0], states.shape[0])) + np.NaN
dmat = np.zeros((states.shape[0], states.shape[0])) + np.NaN
rmat = np.zeros((states.shape[0], states.shape[0])) + np.NaN

lags = range(1, 11)
for i, j in itertools.combinations(range(states.shape[0]), 2):
    #d, r, m = dir_info((states[i]==1).astype(int), (states[j]==1).astype(int), msmlag=2)
    p = inf.MSMProbabilities(msmlag=2, reversible=False, tmat_ck_estimate=False)
    est = inf.TransferEntropy(p)
    est.symmetrized_estimate((states[i]==1).astype(int), (states[j]==1).astype(int))
    dmat[i, j] = est.d #d
    dmat[j, i] = est.r #r
    rmat[i, j] = est.r #r
    
    mmat[i, j] = inf.MutualInfoStationaryDistribution(p).estimate((states[i]==1).astype(int), (states[j]==1).astype(int)).m

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

i = ax[0].imshow(dmat)
ax[0].set_title('transfer entropy(i -> j)')
fig.colorbar(i, ax=ax[0])
i=ax[1].imshow(mmat)
ax[1].set_title('mutual info(i, j)')
fig.colorbar(i, ax=ax[1])

In [ ]:
def correlation(all_a, lag = 0):
    return np.mean(np.vstack([[np.dot(a[i][:, None], a[i+lag][None, :]) for i in range(len(a)-lag)] for a in all_a]), axis = 0)
        
def partial_correlation(a, lag = 0, zero_diags=True):
    #compute correlation matrix
    _corrmat = correlation(a, lag = lag) #np.mean([ np.dot(l[:, None], l[None, :]) for l in a], axis = 0)
    #compute inverse
    ivm = np.linalg.inv(_corrmat)
    #compute normalization from diagonal elements
    divm = np.diag(ivm)
    _norm = np.outer(divm, divm)
    #normalize 
    out = -ivm/np.sqrt(_norm)
    #fill zeros on diagonal?
    if zero_diags:
        np.fill_diagonal(out, 0.)
    return out 

In [ ]:
plt.scatter(mmat.ravel(), correlation([states.T]).ravel())
plt.scatter(dmat.ravel(), correlation([states.T]).ravel(),color='r')

Coupling among neighbors identified; no assymetry in transfer entropy 

### driven Ising model
One spin changes sign as an independent Markov chain and is not influenced by the rest. This example: Spin 10.

In [ ]:
Ising = OneDeeIsing(nspins = 13, coupling = 1, invtemp = 1.0)
states = Ising._state.reshape(-1,1)
energies = [Ising.tot_energy()]

for i in range(10000):
    Ising.driven_sweep(index=10)
    states = np.hstack((states, Ising._state.reshape(-1,1)))
    energies.append(Ising.tot_energy())


fig, ax = plt.subplots(3,1, figsize=(10, 5))

ax[0].plot(energies)
ax[0].set_xlabel('Sweep count')
ax[0].set_ylabel('Total system energy')

ax[1].plot(states.mean(axis=0) )#32, histtype = 'step', log=True)
ax[1].set_ylabel('Effective magnetisation')
ax[1].set_xlabel('Sweep count')

_c = ax[2].imshow(states[:, :100])
ax[2].set_xlabel('Sweep count')
ax[2].set_ylabel('state of spins')
fig.colorbar(_c)

In [ ]:
states.shape

In [ ]:
plt.plot(states[10][:100])
plt.plot(states[11][:100])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

msmlags = range(1, 20, 1)
dis, rdis, mis = [], [], []
for msmlag in msmlags:
    #d, r, m = dir_info((states[10] == 1).astype(int), (states[11] == 1).astype(int), msmlag)
    est = inf.TransferEntropy(inf.MSMProbabilities(msmlag=msmlag, reversible=False, tmat_ck_estimate=False))
    est.symmetrized_estimate((states[10]==1).astype(int), (states[11]==1).astype(int))
    dis.append(est.d)
    rdis.append(est.r)
    mis.append(est.m)
    
ax[0].plot(dis, '.-', label='TE')
ax[0].plot(rdis, '.-', label='rev TE')
ax[0].legend()
ax[0].set_title('TE(driven spin -> neighbor)')
ax[0].set_xlabel('msm lagtime')

dis, rdis, mis = [], [], []
for msmlag in msmlags:
    #d, r, m = dir_info((states[3] == 1).astype(int), (states[4] == 1).astype(int), msmlag)
    est = inf.TransferEntropy(inf.MSMProbabilities(msmlag=msmlag, reversible=False, tmat_ck_estimate=False))
    est.symmetrized_estimate((states[1]==1).astype(int), (states[2]==1).astype(int))
    dis.append(est.d)
    rdis.append(est.r)
    mis.append(est.m)
    
ax[1].plot(dis, '.-', label='TE')
ax[1].plot(rdis, '.-', label='rev TE')
ax[1].legend()
ax[1].set_title('TE(random spin -> neighbor)')
ax[1].set_xlabel('msm lagtime')

In [ ]:
import msmtools, matplotlib

In [ ]:
msmtools.analysis.is_reversible(est.p_estimator.tmat_y), msmtools.analysis.is_reversible(est.p_estimator.tmat_x), msmtools.analysis.is_reversible(est.p_estimator.tmat_xy)

In [ ]:
msmtools.analysis.statdist(est.p_estimator.tmat_xy)[None, :]

In [ ]:
plt.imshow(msmtools.analysis.statdist(est.p_estimator.tmat_xy)[:, None] * est.p_estimator.tmat_xy, norm=matplotlib.colors.LogNorm())
plt.colorbar()

In [ ]:
est.d, est.r, est.m

In [ ]:
mmat = np.zeros((states.shape[0], states.shape[0])) + np.NaN
dmat = np.zeros((states.shape[0], states.shape[0])) + np.NaN
rmat = np.zeros((states.shape[0], states.shape[0])) + np.NaN

for i, j in itertools.combinations(range(states.shape[0]), 2):
    #d, r, m = dir_info((states[i]==1).astype(int), (states[j]==1).astype(int), msmlag=10)
    est = inf.TransferEntropy(inf.MSMProbabilities(msmlag=1, reversible=False, tmat_ck_estimate=False))
    est.symmetrized_estimate((states[i]==1).astype(int), (states[j]==1).astype(int))
    
    dmat[i, j] = est.d
    rmat[i, j] = est.r
    dmat[j, i] = est.r
    
    
    mmat[i, j] = inf.MutualInfoStationaryDistribution(est.p_estimator).estimate((states[i]==1).astype(int), (states[j]==1).astype(int)).m

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

i = ax[0].imshow(dmat, cmap='gist_heat_r')
ax[0].set_title('transfer entropy(i -> j)')
fig.colorbar(i, ax=ax[0])
i=ax[1].imshow(mmat, cmap='gist_heat_r')
ax[1].set_title('mutual info(i, j)')
fig.colorbar(i, ax=ax[1])

In [ ]:
cutoff = .015
a = np.zeros_like(dmat)
a[np.isfinite(rmat)] = dmat[np.isfinite(rmat)] - rmat[np.isfinite(rmat)]
plt.imshow(abs(a) > cutoff)

In [ ]:
print('transfer entropy between spins with significant directionality:\n')
for pair in np.where(abs(a) > cutoff):
    print('te(' + str(pair[0]) + '->' + str(pair[1]) + ')=' + str(np.round(dmat[tuple(pair)], 3)) + \
         ';\tte(' + str(pair[1]) + '->' + str(pair[0]) + ')=' + str(np.round(rmat[tuple(pair)], 3)))

Can identify that spin 10 is not "causaly influenced" by any other spin. Only significant directionality from spin 10 to its nearest neighbors.

In [ ]:
plt.imshow(correlation([states.T], lag=10))#, vmin=-1, vmax=1)
plt.colorbar()